In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import requests
import seaborn as sns
import itertools
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn import ensemble
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score

In [2]:
#url='https://www.kaggle.com/kemical/kickstarter-projects/downloads/ks-projects-201801.csv'
#df=pd.read_csv(url, header=None)

In [3]:
df = pd.read_csv('ks-projects-201801.csv',error_bad_lines=False)

b'Skipping line 20454: expected 15 fields, saw 19\nSkipping line 27266: expected 15 fields, saw 23\nSkipping line 47738: expected 15 fields, saw 19\nSkipping line 54556: expected 15 fields, saw 23\n'
b'Skipping line 75000: expected 15 fields, saw 17\nSkipping line 95465: expected 15 fields, saw 21\nSkipping line 102283: expected 15 fields, saw 26\nSkipping line 129576: expected 15 fields, saw 26\n'
b'Skipping line 170525: expected 15 fields, saw 16\nSkipping line 177348: expected 15 fields, saw 26\n'
b'Skipping line 204638: expected 15 fields, saw 19\nSkipping line 211470: expected 15 fields, saw 26\nSkipping line 218306: expected 15 fields, saw 16\nSkipping line 252430: expected 15 fields, saw 17\nSkipping line 259262: expected 15 fields, saw 26\n'
b'Skipping line 266098: expected 15 fields, saw 18\nSkipping line 300233: expected 15 fields, saw 17\nSkipping line 307065: expected 15 fields, saw 27\nSkipping line 313893: expected 15 fields, saw 19\n'
b'Skipping line 348033: expected 15 

https://www.kaggle.com/kromel/kickstarter-successful-vs-failed/data

In [4]:
df=df.sample(1000, random_state=32)

In [5]:
df.shape

(1000, 15)

In [6]:
df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
711859,292924641,"NextGen, Use any platform on any devive, rules...",Software,Technology,SEK,2014-11-04,50000,2014-10-21 15:14:46,0,failed,0,SE,0,0.00,6726.22
443026,1865145137,Cassidy Jones Adventures: Paperback Books For ...,Fiction,Publishing,USD,2012-07-12,1000,2012-06-28 01:35:01,1505,successful,43,US,1505,1505.00,1000.00
305960,1549399998,· Guenda · products for connecting with nature...,Embroidery,Crafts,MXN,2017-12-25,60000.00,2017-11-25 15:56:14,60520.00,successful,84,MX,112.64,3067.57,3041.21
56190,1111918633,The ABCs of Nightmares,Children's Books,Publishing,CAD,2016-11-24,8500.00,2016-10-24 01:05:08,2764.00,failed,49,CA,0,2048.77,6300.50
867977,7.40164e+08,The Toughest,Documentary,Film & Video,USD,2016-04-02,2295.00,2016-03-03 15:46:40,0.00,failed,0,US,0,0.00,2295.00


In [7]:
df.dtypes

ID                   object
name                 object
category             object
main_category        object
currency             object
deadline             object
goal                 object
launched             object
pledged              object
state                object
backers              object
country              object
usd pledged          object
usd_pledged_real    float64
usd_goal_real       float64
dtype: object

# Data Cleaning

### Nulls

In [8]:
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
df_missingData = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
display(df_missingData.head(20))
df.isnull().sum().max()

,Total,Percent
usd pledged,8,0.008
usd_goal_real,0,0.000
usd_pledged_real,0,0.000
country,0,0.000
backers,0,0.000
state,0,0.000
pledged,0,0.000
launched,0,0.000
goal,0,0.000
deadline,0,0.000


8

In [9]:
dfNoNulls = df.copy()
dfNoNulls.dropna(inplace=True)
print ("Before Dropping Nulls: ", df.shape[0])
print ("After Dropping Nulls: ", dfNoNulls.shape[0])
print ("Rows Removed: ", df.shape[0] - dfNoNulls.shape[0])
print ("Percent Removed: {:.0%}".format(1-(dfNoNulls.shape[0])/df.shape[0]))   

Before Dropping Nulls:  1000
After Dropping Nulls:  992
Rows Removed:  8
Percent Removed: 1%


### "usd pledged" Column: Bad Formatting

In [10]:
dfDropPldg = dfNoNulls.copy()
for row in dfDropPldg.iterrows():
    if str(row[1]['usd pledged']).count('.') > 1:
        print (row)
        dfDrop = dfDropPldg.drop(row[0])
        break;
# Use a comprehension for this

In [11]:
dfDropCntry = dfDropPldg.copy()
dfDropCntry = dfDropCntry.drop(dfDropCntry[(dfDropCntry['country'] != 'US')].index)

In [12]:
dfDropCat = dfDropCntry.copy()
dfDropCat = dfDropCat.drop('category',axis=1)

In [13]:
dfDropState = dfDropCat.copy()
dropVals = ['live','suspended','canceled','fnceled']
for dv in dropVals:
    dfDropState = dfDropState.drop(dfDropState[(dfDropState['state'] == dv)].index)

### Convert to Numeric

In [14]:
dfNum = dfDropState.copy()
numCols = ['goal','pledged','backers','usd pledged']
for c in numCols:
    dfNum[c] = dfNum[c].apply(pd.to_numeric)

In [15]:
dfNum.describe()

,goal,pledged,backers,usd pledged,usd_pledged_real,usd_goal_real
count,7.000000e+02,700.000000,700.000000,700.000000,700.000000,7.000000e+02
mean,2.324955e+04,8470.745529,120.361429,6414.505000,8470.745529,2.324955e+04
std,1.286013e+05,31772.481735,492.706359,27647.878257,31772.481735,1.286013e+05
min,1.000000e+00,0.000000,0.000000,0.000000,0.000000,1.000000e+00
25%,2.000000e+03,81.000000,3.000000,35.000000,81.000000,2.000000e+03
50%,5.000000e+03,1102.500000,20.000000,802.000000,1102.500000,5.000000e+03
75%,1.500000e+04,5208.500000,71.000000,4067.750000,5208.500000,1.500000e+04
max,3.000000e+06,588903.000000,9677.000000,588903.000000,588903.000000,3.000000e+06


# Encoding

In [16]:
sorted(dfNum.state.unique())

['failed', 'successful']

In [17]:
dfEnc = pd.DataFrame()
dfEnc = dfNum.copy()
dfEnc['state'] = dfNum.state.astype('category')
dfEnc['state_code'] = np.where(dfEnc['state'].str.contains('successful'),1,0)

In [18]:
dfEnc = pd.get_dummies(dfEnc, columns=['main_category'], prefix = ['cat'])

In [19]:
dfEnc.shape

(700, 29)

In [20]:
dfEnc.reset_index(inplace=True, drop=True)

# Set Data and Target

In [21]:
contCols = ['goal','pledged','backers','usd pledged', 'usd_pledged_real', 'usd_goal_real']

In [22]:
dataCols = []
dataCols += contCols
dataCols += [col for col in dfEnc.columns if 'cat_' in col]

In [23]:
dataCols

['goal',
 'pledged',
 'backers',
 'usd pledged',
 'usd_pledged_real',
 'usd_goal_real',
 'cat_Art',
 'cat_Comics',
 'cat_Crafts',
 'cat_Dance',
 'cat_Design',
 'cat_Fashion',
 'cat_Film & Video',
 'cat_Food',
 'cat_Games',
 'cat_Journalism',
 'cat_Music',
 'cat_Photography',
 'cat_Publishing',
 'cat_Technology',
 'cat_Theater']

In [24]:
y=dfEnc['state_code']
X=dfEnc[dataCols]

In [25]:
X.head()

,goal,pledged,backers,usd pledged,usd_pledged_real,usd_goal_real,cat_Art,cat_Comics,cat_Crafts,cat_Dance,...,cat_Fashion,cat_Film & Video,cat_Food,cat_Games,cat_Journalism,cat_Music,cat_Photography,cat_Publishing,cat_Technology,cat_Theater
0,1000.0,1505.0,43.0,1505.0,1505.0,1000.0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2295.0,0.0,0.0,0.0,0.0,2295.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,2000.0,2001.0,21.0,2001.0,2001.0,2000.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,2500.0,60.0,3.0,60.0,60.0,2500.0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,3000.0,360.0,12.0,360.0,360.0,3000.0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [26]:
print (y.shape)
y.unique()

(700,)


array([1, 0])

In [27]:
rdgLr1 = LogisticRegression(C=.03, penalty='l2')
print(rdgLr1)
cross_val_score(rdgLr1, X,y, cv=10)

LogisticRegression(C=0.03, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


array([ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  0.98550725])

In [28]:
rdgLr2 = LogisticRegression(C=1000, penalty='l2')
print(rdgLr2)
cross_val_score(rdgLr2, X,y, cv=10)

LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


array([ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  0.98550725])

In [29]:
rdgLr3 = linear_model.RidgeClassifier(alpha=.03)
cross_val_score(rdgLr3, X, y, cv=10)

array([ 0.74647887,  0.81690141,  0.62857143,  0.58571429,  0.75714286,
        0.7       ,  0.62857143,  0.67142857,  0.62318841,  0.68115942])

In [30]:
rdgLr3 = linear_model.RidgeClassifier(alpha=1000)
cross_val_score(rdgLr3, X, y, cv=10)

array([ 0.54929577,  0.63380282,  0.61428571,  0.61428571,  0.64285714,
        0.71428571,  0.62857143,  0.64285714,  0.5942029 ,  0.62318841])